### The purpose of this notebook will be to modify the test data to prepare it for the model.

I'll be reusing the code from the "Data Wrangling" notebook to modify the test data similarly to the training data.

In [1]:
# Import pandas and numpy
import pandas as pd
import numpy as np

# Import LabelEncoder from sklearn.preprocessing
from sklearn.preprocessing import LabelEncoder

In [2]:
# Import the test_data
test_df = pd.read_csv('./Raw_Data/application_test.csv', encoding='utf-8')

# Import the pre-merge training data
train_df = pd.read_csv('.\Intermediate_Data\\intermediate_train.csv', encoding='utf-8', index_col=0)

# Import the intermediate bureau data
bureau_df = pd.read_csv('.\Intermediate_Data\\cleaned_bureau.csv', encoding='utf-8',index_col=0)

# Import the intermediate credit data
credit_df = pd.read_csv('.\Intermediate_Data\\cleaned_credit.csv', encoding='utf-8', index_col=0)

# Import the merge intermediate previous applications and installment adata
prev_instal_df = pd.read_csv('.\Intermediate_Data\\cleaned_prev_instal.csv', encoding='utf-8', index_col=0)

# Import the merged training data for comparison
merged_train_df = pd.read_csv('./Intermediate_Data/merged_train.csv', encoding='utf-8', index_col=0)

In [3]:
# Perform the same feature engineering as training data
EXT_SOURCE = ['EXT_SOURCE_3', 'EXT_SOURCE_2', 'EXT_SOURCE_1']

DOCUMENTS = ['FLAG_DOCUMENT_2', 'FLAG_DOCUMENT_3', 'FLAG_DOCUMENT_4', 'FLAG_DOCUMENT_5', 
             'FLAG_DOCUMENT_6', 'FLAG_DOCUMENT_7', 'FLAG_DOCUMENT_8', 'FLAG_DOCUMENT_9',
             'FLAG_DOCUMENT_10', 'FLAG_DOCUMENT_11', 'FLAG_DOCUMENT_12', 'FLAG_DOCUMENT_13',
             'FLAG_DOCUMENT_14', 'FLAG_DOCUMENT_15', 'FLAG_DOCUMENT_16', 'FLAG_DOCUMENT_17',
             'FLAG_DOCUMENT_18', 'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20', 'FLAG_DOCUMENT_21']

ENQUIRIES = ['AMT_REQ_CREDIT_BUREAU_HOUR', 'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK',
             'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT', 'AMT_REQ_CREDIT_BUREAU_YEAR']

# Define a valid mobile column. This will account if a mobile phone was provided and if it was
# reachable
test_df['VALID_MOBILE'] = (test_df['FLAG_MOBIL'] + test_df['FLAG_CONT_MOBILE']) == 2

# Define a new column for quality of the region and quality of housing relative to region
# I'll put a higher weight on the quality of the region, and a lower weight on the relative rating (3.33 - 10)
test_df['HOUSING'] = (test_df['REGION_RATING_CLIENT'] * 3) + (test_df['REGION_RATING_CLIENT_W_CITY'] / 3)

# Define the new documents column. Take the sum to count how many documents were provided.
test_df['DOCUMENTS'] = test_df[DOCUMENTS].sum(1)

# Define the new enquiries column. Take the sum to count total enquiries to the Credit Bureau over
# the past year.
test_df['ENQUIRIES'] = test_df[ENQUIRIES].sum(1)

In [4]:
# Reindex the test data with columns in the training data
new_test = test_df.reindex(train_df.columns, axis=1)

# Remove the TARGET column
new_test.drop('TARGET', axis=1, inplace=True)

In [5]:
# Let's define a function that takes a dataframe and returns all the columns with null values
def NULL_COLUMNS(df):
    '''Returns all the columns with null values'''
    null_list = []
    for col in df:
        if sum(df[col].isnull()) != 0:
            null_list.append(col)
    return null_list

# Let's check which columns have null values in the training data
print (NULL_COLUMNS(new_test))

['AMT_ANNUITY', 'NAME_TYPE_SUITE', 'OWN_CAR_AGE', 'DEF_30_CNT_SOCIAL_CIRCLE', 'EXT_SOURCE_3', 'EXT_SOURCE_2', 'EXT_SOURCE_1']


In [6]:
# Fill the null values accordingly

# Fill AMT_ANNUITY with 0's
new_test['AMT_ANNUITY'].fillna(new_test['AMT_ANNUITY'].median(), inplace=True)

# Fill AMT_GOODS_PRICE with 0's
new_test['AMT_GOODS_PRICE'].fillna(new_test['AMT_GOODS_PRICE'].median(), inplace=True)

# Convert NAME_TYPE_SUITE to a binary
new_test['NAME_TYPE_SUITE'].fillna(value='Unaccompanied', inplace=True)
suite_convert = (lambda x: 0 if str(x) == 'Unaccompanied' else 1)
new_test['NAME_TYPE_SUITE'] = train_df['NAME_TYPE_SUITE'].apply(suite_convert)

# Fill OWN_CAR_AGE with 0's
new_test['OWN_CAR_AGE'].fillna(0, inplace=True)

# Fill DEF_30_CNT_SOCIAL_CIRCLE with -1's
new_test['DEF_30_CNT_SOCIAL_CIRCLE'].fillna(-1, inplace=True)

for EXT_SOURCE in NULL_COLUMNS(new_test):
    new_test[EXT_SOURCE].fillna(new_test[EXT_SOURCE].median(), inplace=True)

In [7]:
# Sanity check to make sure there are no more null values
assert NULL_COLUMNS(new_test) == []

In [8]:
# Address the outliers and odd values similarly

# Replace the 365243's with 0's
new_test['DAYS_EMPLOYED'].replace(to_replace=365243, value=0, inplace=True)

# Replace the extreme value of 34.0 with 9.0
new_test.loc[train_df['SK_ID_CURR'] == 272071, 'DEF_30_CNT_SOCIAL_CIRCLE'] = 9.0

# Replace the extreme value of 262.0 with 9.0
new_test.loc[train_df['SK_ID_CURR'] == 377322, 'ENQUIRIES'] = 32.0

# Replace the highest value with the second highest value.
new_test.loc[train_df['SK_ID_CURR'] == 377322, 'AMT_INCOME_TOTAL'] = 18000090.0

In [9]:
# Create the domain features as seen i

# Create the variable PAYMENT_RATE
new_test['PAYMENT_RATE'] = new_test['AMT_ANNUITY'] / new_test['AMT_CREDIT']

# Create the variable ANNUITY_INCOME_PERC
new_test['ANN_INCOME_PERC'] = new_test['AMT_ANNUITY'] / new_test['AMT_INCOME_TOTAL']

In [10]:
# Sanity check

# Test data should have all columns in train data besides target, and an additional 2 defined columns here
assert (len(new_test.columns) == (len(train_df.drop('TARGET', axis=1).columns) + 2))

#### Merging Datasets

In [11]:
# Merge the test data and bureau data
test_bureau = new_test.merge(bureau_df, how='left', on='SK_ID_CURR')

# Sanity check to make sure no unique SK_ID_CURR from the training data is lost
assert new_test['SK_ID_CURR'].nunique() == test_bureau['SK_ID_CURR'].nunique()

# Sanity check to check if all columns were added
assert len(test_bureau.columns) == (len(new_test.columns) + len(bureau_df.columns) - 1)

In [12]:
# Check for null values
NULL_COLUMNS(test_bureau)

['LAST_ONGOING_APP',
 'TOTAL_ONGOING',
 'ONGOING_CREDIT',
 'TOTAL_CLOSED',
 'CLOSED_CREDIT']

In [13]:
# Fill np.nan in LAST_ONGOING_APP with -1's
test_bureau['LAST_ONGOING_APP'].fillna(-1, inplace=True)

# Fill np.nan in TOTAL_ONGOING with 0's
test_bureau['TOTAL_ONGOING'].fillna(0, inplace=True)

# Fill np.nan in ONGOING_CREDIT with 0's
test_bureau['ONGOING_CREDIT'].fillna(0, inplace=True)

# Fill np.nan in TOTAL_CLOSED  with 0's
test_bureau['TOTAL_CLOSED'].fillna(0, inplace=True)

# Fill np.nan in CLOSED_CREDIT with 0's
test_bureau['CLOSED_CREDIT'].fillna(0, inplace=True)

In [14]:
# Sanity check to make sure there are no more null values
assert NULL_COLUMNS(test_bureau) == []

In [15]:
# Merge the test_bureau with credit_df using a left merge
Tr_Br_Cr = test_bureau.merge(credit_df, how='left', on='SK_ID_CURR')

# Sanity check to make sure no unique SK_ID_CURR from the training data is lost
assert new_test['SK_ID_CURR'].nunique() == Tr_Br_Cr['SK_ID_CURR'].nunique()

# Sanity check to check if all columns were added
assert len(Tr_Br_Cr.columns) == (len(test_bureau.columns) + len(credit_df.columns) - 1)

In [16]:
# Check for null values
NULL_COLUMNS(Tr_Br_Cr)

['MONTHS_BALANCE', 'SK_DPD', 'NET_PAID']

In [17]:
# Fill the np.nan's in MONTHS_BALANCE with 0's
Tr_Br_Cr['MONTHS_BALANCE'].fillna(0, inplace=True)

# Fill the np.nan's in SK_DPD with 0's
Tr_Br_Cr['SK_DPD'].fillna(0, inplace=True)

# Fill the np.nan's in NET_PAID with 0's
Tr_Br_Cr['NET_PAID'].fillna(0, inplace=True)

In [18]:
# Sanity check to make sure there are no more null values
assert NULL_COLUMNS(Tr_Br_Cr) == []

In [19]:
# Rename the AMT_CREDIT column to PREV_CREDIT to avoid overlapping columns
prev_instal_df.columns = ['SK_ID_CURR', 'PREV_CREDIT', 'DAYS_DECISION',
                         'DAYS_TERMINATION', 'NET_PAYMENT', 'PAYMENT_TIME']

In [20]:
# Merge the train_bureau with credit_df using a left merge
merged_test = Tr_Br_Cr.merge(prev_instal_df, how='left', on='SK_ID_CURR')

# Sanity check to make sure no unique SK_ID_CURR from the training data is lost
assert new_test['SK_ID_CURR'].nunique() == merged_test['SK_ID_CURR'].nunique()

# Sanity check to check if all columns were added
assert len(merged_test.columns) == (len(Tr_Br_Cr.columns) + len(prev_instal_df.columns) - 1)

In [21]:
# Check for null values
NULL_COLUMNS(merged_test)

['PREV_CREDIT',
 'DAYS_DECISION',
 'DAYS_TERMINATION',
 'NET_PAYMENT',
 'PAYMENT_TIME']

In [22]:
# Fill the np.nan's in PREV_CREDIT with 0's
merged_test['PREV_CREDIT'].fillna(0, inplace=True)

# Fill the np.nan's in DAYS_DECISION with 0's
merged_test['DAYS_DECISION'].fillna(0, inplace=True)

# Fill the np.nan's in DAYS_TERMINATION with 0's
merged_test['DAYS_TERMINATION'].fillna(0, inplace=True)

# Fill the np.nan's in NET_PAYMENT with 0's
merged_test['NET_PAYMENT'].fillna(0, inplace=True)

# Fill the np.nan's in PAYMENT_TIME with 0's
merged_test['PAYMENT_TIME'].fillna(0, inplace=True)

In [23]:
# Drop the AMT_ANNUITY and AMT_GOODS_PRICE columns
# merged_test.drop(['AMT_ANNUITY', 'AMT_GOODS_PRICE'], axis=1, inplace=True)

In [24]:
# Test data should have all columns in train data besides target
assert (list(merged_test.columns) == list(merged_train_df.drop('TARGET', axis=1).columns))

#### Encoding

In [25]:
encoding_test = merged_test.copy()

In [26]:
# Create new lists with binary and non-binary categories. Leave out the SK_ID_CURR and TARGET from the columns

# Define the binary categories
BINARY_CAT = ['NAME_CONTRACT_TYPE', 'FLAG_OWN_REALTY', 'NAME_TYPE_SUITE', 'REG_REGION_NOT_LIVE_REGION', 'VALID_MOBILE']

# Define the non-binary categories
NON_BIN_CAT = ['NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE']

# Instantiate the LabelEncoder
labEncode = LabelEncoder()

# Convert the dtype category columns in BINARY_CAT to numeric data using LabelEncoder
for col in BINARY_CAT:
    if encoding_test[col].dtype != 'int64':
        encoding_test[col] = labEncode.fit_transform(encoding_test[col])
        
# Convert all column sin NON_BIN_CAT to numeric data using LabelEncoder
for col in NON_BIN_CAT:
    encoding_test[col] = labEncode.fit_transform(encoding_test[col])

In [27]:
# Drop the DAYS_DECISION column
encoding_test.drop(['DAYS_DECISION', 'AMT_GOODS_PRICE', 'CNT_CHILDREN'], axis=1, inplace=True)

# Sanity check to make sure the column has been dropped
assert 'DAYS_DECISION' not in encoding_test.columns

In [28]:
# Sanity checks

# Ensure no rows were lost
assert len(encoding_test) == len(merged_test)

# Ensure no additional columns were removed besides DAYS_DECISIONS
assert len(encoding_test.columns) == len(merged_test.columns) - 3

In [29]:
# Write the dataframe to a csv for use in another notebook
encoding_test.to_csv('./Intermediate_Data/encoded_test.csv', encoding='utf-8')